## Vanilla MobileNetV3 Implementation

**AIM: Build and train an image classifier to detect images from different animal species using a Custom MobileNet Model in TensorFlow.**

### Objectives

- Data visualisation
- Data preprocessing and image augmentation
- Replicate the MobileNetV3 architecture for model development.
- Compile and train the model
- Add early stopping callback
- Save and load the model
- Model evaluation.
- Make predictions on new data using the trained model.

### Pre-requisite
- Google collaboratry or Jupyter Notebook
- animal-image-classification-dataset
- TensorFlow2

In [1]:
# Import basic libraries
import os
import sys
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib

2026-02-15 11:10:51.068170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771150251.147313    8678 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771150251.169060    8678 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771150251.324589    8678 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771150251.324610    8678 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771150251.324611    8678 computation_placer.cc:177] computation placer alr

In [2]:
# Set seed for reproducibility

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# Check GPU availability
!nvidia-smi

Sun Feb 15 11:10:53 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.126.09             Driver Version: 580.126.09     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 4000                Off |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              6W /  110W |       6MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
gpus= tf.config.list_physical_devices()

In [5]:

gpus

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
logical_devices = tf.config.list_logical_devices()
logical_devices

I0000 00:00:1771150254.392470    8678 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6624 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:01:00.0, compute capability: 7.5


[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [7]:
# Check tenorflow version
print("TensorFlow Version", tf.__version__)

TensorFlow Version 2.19.0


In [8]:
## Set the base path
base_dir = "../dataset/animal_image_classification_dataset"
base_dir = pathlib.Path(base_dir)
base_dir

PosixPath('../dataset/animal_image_classification_dataset')

In [9]:
# Train directory
train_dir = base_dir / "Training Data" / "Training Data"
train_dir

PosixPath('../dataset/animal_image_classification_dataset/Training Data/Training Data')

In [10]:
# Validation directory
validation_dir = base_dir / "Validation Data" / "Validation Data"
validation_dir

PosixPath('../dataset/animal_image_classification_dataset/Validation Data/Validation Data')

In [11]:
## Set Hyperparameters

IMAGE_HEIGHT, IMAGE_WIDTH = 128, 128
BATCH_SIZE = 64
EPOCHS = 300

In [12]:
# Load the training dataset

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    seed=SEED,
)

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/animal_image_classification_dataset/Training Data/Training Data'

In [ ]:
# Load the validation dataset

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    seed=SEED,
)

In [ ]:
# Get the class names
class_names = train_dataset.class_names
class_names

In [ ]:
# Get the total number of classes
num_classes = len(class_names)
num_classes

In [ ]:
# Sanity check

for images, labels in train_dataset.take(1):
    fixed_images = images.numpy()
    fixed_labels = labels.numpy()


# Visualisations
# No matter how many times you run this cell, the images won change because of teh above

plt.figure(figsize=(12, 12))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(fixed_images[i].astype("uint8"))
    plt.title(class_names[fixed_labels[i]])
    plt.axis("off")

In [ ]:
# Performance optimization

### Vanilla MobileNetV3 Implementation

In [ ]:
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH) + (3, )
INPUT_SHAPE

In [ ]:
*

In [ ]:
inputs = tf.keras.layers.Input(shape=INPUT_SHAPE)

# STEM CONVOLUTION (Initial Layer)
x = tf.keras.layers.Conv2D(filters=32,
                                kernel_size=(3, 3),
                                strides=2,
                                padding="same",
                                use_bias=False)(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

# BLOCK 1 (Expansion = 1)
block_input = x

x = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                    strides=1,
                                    padding="same",
                                    use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.Conv2D(filters=16,
                           kernel_size=1,
                           padding="same",
                           use_bias=False)(x)
x = tf.keras.layers.BatchNormalization()(x)

# NOTE: No residual here because channel mismatch


# BLOCK 2 (Expansion = 6, Stride = 2)
block_input = x

x = tf.keras.layers.Conv2D(filters=16*6,
                           kernel_size=(1, 1),
                           padding="same",
                           use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                    strides=2,
                                    padding="same",
                                    use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.Conv2D(filters=24,
                           kernel_size=1,
                           padding="same",
                           use_bias=False)(x)
x = tf.keras.layers.BatchNormalization()(x)


# BLOCK 3 (Expansion = 6, Stride = 1, Residual)
block_input = x

x = tf.keras.layers.Conv2D(filters=24*6,
                           kernel_size=(1, 1),
                           padding="same",
                           use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                    strides=1,
                                    padding="same",
                                    use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.Conv2D(filters=24,
                           kernel_size=1,
                           padding="same",
                           use_bias=False)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Add()([block_input, x])


# BLOCK 4 (Expansion = 6, Stride = 2)
block_input = x

x = tf.keras.layers.Conv2D(filters=24*6,
                           kernel_size=(1, 1),
                           padding="same",
                           use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                    strides=2,
                                    padding="same",
                                    use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.Conv2D(filters=32,
                           kernel_size=1,
                           padding="same",
                           use_bias=False)(x)
x = tf.keras.layers.BatchNormalization()(x)


# BLOCK 5 (Expansion = 6, Residual)
block_input = x

x = tf.keras.layers.Conv2D(filters=32*6,
                           kernel_size=(1, 1),
                           padding="same",
                           use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                    strides=1,
                                    padding="same",
                                    use_bias=False)(block_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)

x = tf.keras.layers.Conv2D(filters=32,
                           kernel_size=1,
                           padding="same",
                           use_bias=False)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Add()([block_input, x])


# Final Conv Layer
x = tf.keras.layers.Conv2D(filters=1280,
                           kernel_size=1,
                           padding="same",
                           use_bias=False)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU(max_value=6)(x)


# Classification Head

x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.summary()

In [ ]:
# Compile Model
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss=loss_function,
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [ ]:
# Configure Callbacks

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/vanilla_mobilenet_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=1,
    restore_best_weights=True
)

reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=5,
    factor=0.3,
    verbose=1
)

callbacks = [model_checkpoint, early_stopping, reduce_learning_rate]


In [ ]:
# Train the Model to learn patterns from the image

history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=EPOCHS,
                    callbacks=callbacks)

In [ ]:
def plot_learning_curves(history):
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs_range = range(len(acc))


    plt.figure(figsize=(18, 7))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label="Training Accuracy")
    plt.plot(epochs_range, val_acc, label="Validation Accuracy")
    plt.legend()
    plt.title("Accuracy")

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label="Training Loss")
    plt.plot(epochs_range, val_loss, label="Validation Loss")
    plt.legend()
    plt.title("Loss")

    plt.show()


In [ ]:
loss, accuracy = model.evaluate(validation_dataset)

print(f"Model Loss: {loss:.2f}")
print(f"Model Accuracy: {accuracy:.2f}")